In [1]:
%cd /raid6/home/yokoyama/har-for-or/
import os
import sys
from glob import glob
import pickle

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

sys.path.append(".")
from src.utils import yaml_handler, vis

/raid6/home/yokoyama/har-for-or


/raid6/home/yokoyama/har-for-or/.venv/lib/python3.10/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [2]:
# dataset_num = 5
# video_num = 11

# # load preds
# data_root = f"../datasets/dataset{dataset_num:02d}/train/{video_num:02d}"
# paths = glob(os.path.join(data_root, f"pred_{model_type}", "*"))
# results = []
# for path in paths:
#     with open(path, "rb") as f:
#         results.append(pickle.load(f))

# # count labels
# labels = {}
# for result in results:
#     key = result["key"]
#     video_num, n_frame, _id = key.split("_")
#     key = f"{video_num}_{_id}"

#     if key not in labels:
#         labels[key] = []
#     labels[key].append(result["label"])

# # get most frequent labels
# data = []
# for key, val in labels.items():
#     unique, counts = np.unique(val, return_counts=True)
#     maxidx = np.argmax(counts)
#     label = unique[maxidx].item()
#     data.append([key, label])

# # save
# path = f"../datasets/dataset{dataset_num:02d}/annotation/role_train_{video_num}.txt"
# np.savetxt(path, data, fmt="%s", delimiter=" ")

In [ ]:
# test
model_type = "sqvae"
dataset_num = 5
if dataset_num == 3:
    v = 0
elif dataset_num == 4:
    v = 1
elif dataset_num == 5:
    v = 2
checkpoint_dir = f"models/individual/{model_type}/version_{v}"


# load classes
path = f"../datasets/dataset{dataset_num:02d}/annotation/classes.txt"
classes = np.loadtxt(path, str, usecols=0, delimiter=",")
classes = [c.title() for c in classes]

# load annotation
path = glob(f"../datasets/dataset{dataset_num:02d}/annotation/*test*.txt")[0]
video_num = int(os.path.basename(path).split(".")[0].split("_")[2])
annotations = np.loadtxt(path, str, skiprows=1, delimiter=" ")

# load config
config = yaml_handler.load(f"{checkpoint_dir}/individual-{model_type}.yaml")
seq_len = config.seq_len
stride = config.stride
if config.mask_leg:
    n_pts = 13 + 2
else:
    n_pts = 17 + 2

# load preds
data_root = f"../datasets/dataset{dataset_num:02d}/test/{video_num:02d}"
paths = glob(os.path.join(data_root, f"pred_{model_type}", "*"))
results = []
for path in paths:
    with open(path, "rb") as f:
        results.append(pickle.load(f))

img_dir = f"../datasets/dataset{dataset_num:02d}/images"
os.makedirs(img_dir, exist_ok=True)

In [6]:
data_dirs = glob(f"../datasets/dataset{dataset_num:02d}/train/*/") + glob(
    f"../datasets/dataset{dataset_num:02d}/test/*/"
)

# sort by video_num
data_dirs = sorted(data_dirs, key=lambda x: int(os.path.basename(os.path.dirname(x))))

In [7]:
for data_dir in tqdm(data_dirs):
    video_num = os.path.basename(os.path.dirname(data_dir))

    # load preds
    paths = glob(os.path.join(data_dir, f"pred_{model_type}", "*"))
    results = []
    for path in paths:
        with open(path, "rb") as f:
            results.append(pickle.load(f))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:34<00:00,  2.29s/it]


In [8]:
# print("clustering label")
# X = np.array(latent_features["ze"]).reshape(-1, config.latent_ndim * n_pts * 2)
# labels = np.array(latent_features["label"])
# vis.plot_tsne(X, labels, 10, None, True, cmap="tab10")